# DTW alignment on the basis of energy

(suggested by results from MFCC tests in vowel-discri git).

In [ ]:
%matplotlib inline

# generic imports
import numpy as np
import os.path as path



# for feature extraction/storage
import soundfile
import h5features



# for result plots
from scone_phobia import apply_analysis
from scone_phobia.analyses.avg_error import avg_error
import scone_phobia.metadata.add_metadata as add_metadata
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%load_ext Cython

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon June 10 10:14:04 2019

@author: Thomas Schatz
Adapted from https://github.com/librosa

Compute MFCC coefficients.

Steps:
    Waveform -> pre-emphasis -> STFT with Hanning window 25ms + 10ms -> 128 channels mel power-spectrogram
    using area-normalized triangular filters over 0-8000Hz -> to DB (log-compression) -> type II DCT (PCA)
    -> 13 coefficients -> replace 0-th order coefficient with log-energy or drop it (optional)
    -> perform cepstral mean normalization (optional)

Something we did not implement for now: use energy to replace first MFCC coefficient.

Main differences with librosa default are:
    - no padding of time-series to get first frame centered on t=0
    - signal is pre-emphasized
    - defaults: n_MFCC=13, sr=16000, n_fft=400, hop_length=160
    - works from time-series only
    - allows optional dropping or replacement with log-energy of zeroth-order MFCC coefficient
    - allows optional cepstral mean normalization


Requirements: librosa, numpy, scipy

Usage:
    import soundfile
    y, fs = soundfile.read(filename)
    assert fs == 16000
    coefs = mfcc(y)
    # coefs is a 13 by nb_frames numpy array of MFCCs
"""

import numpy as np
import scipy.fftpack
import scipy.signal as sig
from librosa.core.spectrum import power_to_db, stft
from librosa import filters


def pre_emphasize(y):
    b = [1, -.97]
    a = 1
    zi = sig.lfilter_zi(b, a)
    return sig.lfilter(b, a, y, zi=zi)[0]


def log_energy(y, n_fft=400, hop_length=160):
    power_spectrum = np.abs(stft(y, n_fft=n_fft, hop_length=hop_length, center=False))**2
    log_E = 10*np.log10(sum(power_spectrum))  # in dB
    return log_E


def melspectrogram(y=None, sr=16000, n_fft=400, hop_length=160,
                   power=2.0, **kwargs):
    """Compute a mel-scaled spectrogram.

    If a spectrogram input `S` is provided, then it is mapped directly onto
    the mel basis `mel_f` by `mel_f.dot(S)`.

    If a time-series input `y, sr` is provided, then its magnitude spectrogram
    `S` is first computed, and then mapped onto the mel scale by
    `mel_f.dot(S**power)`.  By default, `power=2` operates on a power spectrum.

    Parameters
    ----------
    y : np.ndarray [shape=(n,)] or None
        audio time-series

    sr : number > 0 [scalar]
        sampling rate of `y`

    n_fft : int > 0 [scalar]
        length of the FFT window

    hop_length : int > 0 [scalar]
        number of samples between successive frames.
        See `librosa.core.stft`

    power : float > 0 [scalar]
        Exponent for the magnitude melspectrogram.
        e.g., 1 for energy, 2 for power, etc.

    kwargs : additional keyword arguments
      Mel filter bank parameters.
      See `librosa.filters.mel` for details.

    Returns
    -------
    S : np.ndarray [shape=(n_mels, t)]
        Mel spectrogram
    """
    # Compute a magnitude spectrogram from input
    S = np.abs(stft(y, n_fft=n_fft, hop_length=hop_length, center=False))**power

    # Build a Mel filter
    mel_basis = filters.mel(sr, n_fft, **kwargs)

    return np.dot(mel_basis, S)


def mfcc(y=None, sr=16000, n_mfcc=13, dct_type=2, norm='ortho', 
         zeroth_coef=None, cep_mean_norm=False, **kwargs):
    """Mel-frequency cepstral coefficients (MFCCs)

    Parameters
    ----------
    y     : np.ndarray [shape=(n,)] or None
        audio time series

    sr    : number > 0 [scalar]
        sampling rate of `y`

    n_mfcc: int > 0 [scalar]
        number of MFCCs to return

    dct_type : None, or {1, 2, 3}
        Discrete cosine transform (DCT) type.
        By default, DCT type-2 is used.

    norm : None or 'ortho'
        If `dct_type` is `2 or 3`, setting `norm='ortho'` uses an ortho-normal
        DCT basis.

        Normalization is not supported for `dct_type=1`.

    kwargs : additional keyword arguments
        Arguments to `melspectrogram`, if operating
        on time series input

    Returns
    -------
    M     : np.ndarray [shape=(n_mfcc, t)]
        MFCC sequence
    """

    # pre-emphasize signal
    y = pre_emphasize(y)

    # compute mel-spectrogram
    S = power_to_db(melspectrogram(y=y, sr=sr, **kwargs))

    # compute MFCCs
    coefs = scipy.fftpack.dct(S, axis=0, type=dct_type, norm=norm)[:n_mfcc]

    if zeroth_coef == 'energy':
        # replace 0th order MFCC coef with log energy
        coefs[0, :] = log_energy(y)
    elif zeroth_coef == 'remove':
        coefs = coefs[1:, :]

    if cep_mean_norm:
        # do cepstral mean normalization
        coefs = coefs - np.mean(coefs, axis=0)

    return coefs


In [ ]:
def read_wav(wavefile, onset=None, offset=None):
    data, fs = soundfile.read(wavefile)
    if onset is None:
        onset = 0
    if offset is None:
        offset = len(data)/float(fs)
    times = 1/(2*float(fs)) + np.arange(len(data))/float(fs)
    data = data[(times>=onset) & (times<=offset)]
    return data, fs


def extract_mfcc(wav_folder, segments_file, out_file, **kwargs):
    segments = {}
    with open(segments_file, 'r') as fh:
        for line in fh:
            seg, wav, onset, offset = line.strip().split()
            onset, offset = float(onset), float(offset)
            segments[seg] = wav, onset, offset
    utts, feats, times = [], [], []
    for i, seg in enumerate(segments):
        if i % 100 == 0:
            print("Done {} / {}".format(i, len(segments)))
        wav, onset, offset = segments[seg]
        wavefile = path.join(wav_folder, wav)
        data, fs = read_wav(wavefile, onset=onset, offset=offset)
        assert fs == 16000
        coefs = mfcc.mfcc(data, **kwargs)
        feats.append(coefs.T)
        utts.append(seg)
        times.append(0.0125 + np.arange(coefs.shape[1])*0.01)
    data = h5features.Data(utts, times, feats, check=True)
    with h5features.Writer(out_file) as fh:
        fh.write(data, '/features')

In [ ]:
data, fs = soundfile.read('./bras.wav')
assert fs == 16000

## Test ABX discriminability on WSJ corpus

This requires some of the material from the "Early phonetic learning without phonetic categories" paper.


### First extract features of interest and store them in h5features format to allow testing ABX phone discriminability.

In [ ]:
conditions = [('energy', True),  ('energy', False),
              ('remove', False), ('remove', True),
              (None, False), (None, True)]
root = '/Users/admin/Documents/PhD/Data/GPJ_match_WSJ_test/'
for zeroth, norm in conditions[1:]:
    print(zeroth)
    print(norm)
    data = extract_mfcc(root + 'wavs/',
                        root + 'segments.txt',
                        root + 'mfcc/mfcc_{}_{}.features'.format(zeroth, norm),
                        zeroth_coef=zeroth, cep_mean_norm=norm)

## 2. Run WSJ discrimination on features (done on a remote cluster)

## 3. Plot results

In [ ]:
# Loading (or computing if it's the first time) avg_error analysis results with full resamples

mp_folder = '/Users/admin/Documents/PhD/Data/vowel_discri/mp_scores'

analysis = avg_error
df_avg = apply_analysis(analysis, mp_folder,
                        add_metadata=add_metadata.language_register,
                        resampling=False)

In [ ]:
sns.catplot(data=df_avg, kind='bar', y='error', hue='model type', x='contrast type')

Conclusion: for within-speaker phone discrimination:
    - cepstral mean normalization does not have much effect overall (tends to make consonant a bit harder to discriminate and vowel a bit easier) -> do not do it
    - log-energy: big effect unscaled zeroth-order MFCC appears best????
        - Does this have to do with DTW aligning signal based on energy profile due to scale unbalance???
            -> how come there is this unbalance in the first place, isn't this supposed to be a PCA?
                -> maybe log-energy synchronization (DTW or otherwise) + cosine distance on aligned signals without the energy would work very well? (or more generally on signal deconvoluted from pitch+prosody contours)
   
    - Short-term: take normalized DTW with basic MFCC (no log energy or removing of first coefficient) and no cepstral mean normalization. 